## 2021年の沢村賞投手を予測する機械学習モデル

**沢村賞**とは、プロ野球草創期の名投手・沢村栄治選手（読売巨人軍）の功績をたたえ、プロ野球シーズン中に最も好成績をあげた「先発完投型」の投手に贈る賞である。

選考方法は
(1)登板数が25試合以上。
(2)完投数10試合以上。
(3)勝利数が15勝以上。
(4)勝率6割以上。
(5)投球イニング数が200以上。
(6)奪三振の数150以上。
(7)防御率2.50以下。
以上の7項目を参考に選考委員会が受賞者を決定する。
7項目のすべてを満たす必要はないため、最後は選考委員会次第である。

過去の沢村賞投手の傾向から2021年の沢村賞を予測する。
2009年から2021年で一軍で投げた全投手の成績を使用。
2009年～2020年の投手データを機械学習させて、2021年の投手データをテストデータとする。

2021年の沢村賞は「山本由伸」投手であるので機械学習で彼を予測することができたのならば、ひとまず目標達成ということです。

またモデルは「ロジスティック回帰」を使用したものと「LightGBM」を使用したものを作成し、精度を比べる。
これらを選択した理由は、どちらも二値分類によく使われるからです。

In [183]:
import numpy as np
import pandas as pd 
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression

作成したcsvファイルを読み込み、前処理をする。

In [148]:
#csvファイルを読み込む
df = pd.read_csv('sawamura/SawamuraPitcher_total.csv', sep = ',')

#投球回が0の投手は欠損値を持つため除外
dfs = df[df['投球回'] > 0 ]

#防御率,whip,dipsをobjectからfloat64へ
dfs['防御率'] = pd.to_numeric(dfs['防御率'],errors = 'coerce')
dfs['WHIP'] = pd.to_numeric(dfs['WHIP'],errors = 'coerce')
dfs['DIPS'] = pd.to_numeric(dfs['DIPS'],errors = 'coerce')

print(dfs)

        選手名     チーム    防御率  試合  勝利  敗北  セーブ  ホールド     勝率   打者  ...  与死球  奪三振  \
0     涌井　秀章      西武   2.30  27  16   6    0     0  0.727  863  ...    9  199   
1     石川　雅規    ヤクルト   3.54  29  13   7    0     0  0.650  810  ...    6   84   
2     三浦　大輔      横浜   3.32  28  11  11    0     0  0.500  799  ...    4  138   
3     前田　健太      広島   3.36  29   8  14    0     0  0.364  795  ...    3  147   
4     杉内　俊哉  ソフトバンク   2.36  26  15   5    0     0  0.750  764  ...    4  204   
...     ...     ...    ...  ..  ..  ..  ...   ...    ...  ...  ...  ...  ...   
4232   中川　颯   オリックス   0.00   1   0   0    0     0  0.000    5  ...    0    0   
4233  笠井　崇正    DeNA  54.00   1   0   0    0     0  0.000   11  ...    1    1   
4234  山井　大介      中日   0.00   1   0   0    0     0  0.000    1  ...    0    1   
4235   南　昌輝     ロッテ   0.00   1   0   0    0     1  0.000    1  ...    0    1   
4236  武藤　祐太    DeNA   0.00   1   0   0    0     0  0.000    1  ...    0    1   

      失点  自責点  WHIP   DIPS  完投  完封  yea

訓練データとテストデータに分割

In [149]:
#2009~2020年のデータ
df_train = dfs[df['year'] != 2021]
#テストデータとして使う2021年のデータ
df_test = dfs[df['year'] == 2021]

In [84]:
df_train.columns

Index(['選手名', 'チーム', '防御率', '試合', '勝利', '敗北', 'セーブ', 'ホールド', '勝率', '打者', '投球回',
       '被安打', '被本塁打', '与四球', '与死球', '奪三振', '失点', '自責点', 'WHIP', 'DIPS', '完投',
       '完封', 'year', '沢村賞'],
      dtype='object')

In [122]:
df_train

,選手名,チーム,防御率,試合,勝利,敗北,セーブ,ホールド,勝率,打者,...,与死球,奪三振,失点,自責点,WHIP,DIPS,完投,完封,year,沢村賞
0,涌井 秀章,西武,2.30,27,16,6,0,0,0.727,863,...,9,199,57,54,1.12,3.15,11,4,2009,1
1,石川 雅規,ヤクルト,3.54,29,13,7,0,0,0.650,810,...,6,84,81,78,1.16,4.38,3,0,2009,0
2,三浦 大輔,横浜,3.32,28,11,11,0,0,0.500,799,...,4,138,82,72,1.09,4.18,6,1,2009,0
3,前田 健太,広島,3.36,29,8,14,0,0,0.364,795,...,3,147,82,72,1.16,3.56,3,1,2009,0
4,杉内 俊哉,ソフトバンク,2.36,26,15,5,0,0,0.750,764,...,4,204,59,50,1.09,2.97,6,1,2009,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3890,笠井 崇正,DeNA,18.00,1,0,0,0,0,0.000,7,...,0,1,2,2,4.00,4.12,0,0,2020,0
3891,モンティージャ,広島,18.00,1,0,0,0,0,0.000,6,...,0,0,2,2,3.00,3.12,0,0,2020,0
3892,増田 大輝,巨人,0.00,1,0,0,0,0,0.000,3,...,0,0,0,0,1.50,7.62,0,0,2020,0
3893,浦野 博司,日本ハム,0.00,1,0,0,0,0,0.000,1,...,0,1,0,0,0.00,-2.88,0,0,2020,0


In [96]:
df_test

,選手名,チーム,防御率,試合,勝利,敗北,セーブ,ホールド,勝率,打者,...,与死球,奪三振,失点,自責点,WHIP,DIPS,完投,完封,year,沢村賞
3896,山本 由伸,オリックス,1.39,26,18,5,0,0,0.783,736,...,2,206,37,30,0.85,2.10,6,4,2021,1
3897,髙橋 光成,西武,3.78,27,11,9,0,0,0.550,728,...,6,127,81,73,1.26,4.55,0,0,2021,0
3898,柳 裕也,中日,2.20,26,11,6,0,0,0.647,676,...,3,168,47,42,1.01,2.75,2,2,2021,0
3899,森下 暢仁,広島,2.98,24,8,7,0,0,0.533,672,...,3,132,55,54,1.20,3.73,1,1,2021,0
3900,上沢 直之,日本ハム,2.81,24,12,6,0,0,0.667,643,...,4,135,53,50,1.04,3.04,1,0,2021,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4232,中川 颯,オリックス,0.00,1,0,0,0,0,0.000,5,...,0,0,0,0,2.00,3.12,0,0,2021,0
4233,笠井 崇正,DeNA,54.00,1,0,0,0,0,0.000,11,...,1,1,6,6,7.00,23.12,0,0,2021,0
4234,山井 大介,中日,0.00,1,0,0,0,0,0.000,1,...,0,1,0,0,0.00,-2.88,0,0,2021,0
4235,南 昌輝,ロッテ,0.00,1,0,0,0,1,0.000,1,...,0,1,0,0,0.00,-2.88,0,0,2021,0


沢村賞と各々の項目の相関係数を調べる。

In [150]:
df_sawamura = pd.DataFrame(df_train.corr()['沢村賞'])

# 沢村賞と相関係数「0.1」以上の項目名を抽出
X_columns = df_sawamura[df_sawamura['沢村賞'] >= 0.1].index.tolist()

print(df_sawamura[df_sawamura['沢村賞'] >= 0.1])

          沢村賞
勝利   0.215085
打者   0.156745
投球回  0.168651
被安打  0.131443
奪三振  0.193741
完投   0.349535
完封   0.348847
沢村賞  1.000000


相関係数を参考にして説明変数を決定する。この場合は相関係数が0.1以上のものを抽出。
目的変数は沢村賞。
訓練データとテストデータをそれぞれ用意する。

In [184]:
X_train = df_train[['勝利','勝率','打者','投球回','奪三振','失点','自責点', 
                    '完投','完封']]
y_train = df_train[['沢村賞']]


X_test = df_test[['勝利','勝率','打者','投球回','奪三振','失点','自責点',
                  '完投','完封']]
y_test = df_test[['沢村賞']]

**ロジスティクス回帰**を使用して学習させる。
ロジスティック回帰分析は発生確率を予測する手法。

In [152]:
# モデルの学習
model_lr = LogisticRegression(max_iter=1000)# モデルのインスタンスの作成
model_lr.fit(X_train, y_train)# モデルの学習

# テストデータの予測クラス (予測クラス(0 or 1)を返す)
y_pred_lr =model_lr.predict(X_test)
# テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率] を返す)
y_pred_prob_lr = model_lr.predict_proba(X_test)

閾値である0.5を超えたら1と表示、それ以外は0

In [153]:
model_lr.predict(X_test)#閾値は0.5

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

［沢村賞で”ない”事の予測確率,沢村賞で”ある”ことの予測確率］

In [154]:
print(y_pred_prob_lr)

[[6.61525894e-01 3.38474106e-01]
 [9.99841648e-01 1.58352193e-04]
 [9.94204530e-01 5.79546973e-03]
 [9.97503595e-01 2.49640520e-03]
 [9.98015148e-01 1.98485234e-03]
 [9.99371821e-01 6.28178830e-04]
 [9.99973996e-01 2.60039157e-05]
 [9.91874659e-01 8.12534114e-03]
 [9.99832296e-01 1.67703854e-04]
 [9.99902818e-01 9.71816835e-05]
 [9.99915105e-01 8.48949480e-05]
 [9.98230169e-01 1.76983110e-03]
 [9.99649030e-01 3.50969933e-04]
 [9.99653647e-01 3.46352882e-04]
 [9.96467099e-01 3.53290078e-03]
 [9.99523217e-01 4.76782786e-04]
 [9.99219120e-01 7.80880429e-04]
 [9.98233866e-01 1.76613431e-03]
 [9.99529176e-01 4.70824062e-04]
 [9.99948634e-01 5.13660130e-05]
 [9.99506943e-01 4.93057417e-04]
 [9.99846933e-01 1.53067480e-04]
 [9.99676643e-01 3.23357352e-04]
 [9.98841365e-01 1.15863497e-03]
 [9.93055308e-01 6.94469169e-03]
 [9.98685717e-01 1.31428258e-03]
 [9.99619876e-01 3.80124331e-04]
 [9.98925303e-01 1.07469690e-03]
 [9.99923124e-01 7.68758842e-05]
 [9.99851755e-01 1.48244528e-04]
 [9.999256

In [134]:
df_test[0:1]

,選手名,チーム,防御率,試合,勝利,敗北,セーブ,ホールド,勝率,打者,...,与死球,奪三振,失点,自責点,WHIP,DIPS,完投,完封,year,沢村賞
3896,山本 由伸,オリックス,1.39,26,18,5,0,0,0.783,736,...,2,206,37,30,0.85,2.1,6,4,2021,1


閾値である0.5を超える投手はいなかったが沢村賞の確率が最も高い投手は山本由伸であった。その確率は約33.8%である。他の投手と比べると断トツで確率が高かったので閾値を調整すれば、0と1で分類できるであろう。

### ロジスティック回帰モデルの評価

訓練データとテストデータに対する正解率

In [162]:
#訓練データの正解率
print('train score : ', model_lr.score(X_train, y_train))

train score :  0.9976816074188563


In [163]:
#テストのデータの正解率
print('test score : ', model_lr.score(X_test, y_test))

test score :  0.9970674486803519


F値による評価

In [157]:
from sklearn.metrics import f1_score
f_score = f1_score(y_test,y_pred_lr)
print(f_score)

0.0


**LightGBM**によって学習させる。LightGBMはMicrosoftが開発した決定木アルゴリズムに基づいた勾配ブースティングアルゴリズムを扱うための機械学習フレームワークである。最近人気であり、推測精度も高いことで知られる。

In [176]:
import lightgbm as lgb #LightGBM
# モデルの学習
model_lgb = lgb.LGBMClassifier() # モデルのインスタンスの作成
model_lgb.fit(X_train, y_train) # モデルの学習

# テストデータの予測クラス (予測クラス(0 or 1)を返す)
y_pred_lgb = model_lgb.predict(X_test)
# テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率] を返す)
y_pred_prob_lgb = model_lgb.predict_proba(X_test)

In [177]:
model_lgb.predict(X_test)#閾値は0.5

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

［沢村賞で”ない”事の予測確率,沢村賞で”ある”ことの予測確率］

In [178]:
print(y_pred_prob_lgb)

[[5.83943586e-02 9.41605641e-01]
 [9.99999907e-01 9.30576341e-08]
 [9.99819812e-01 1.80187732e-04]
 [9.99999391e-01 6.09483027e-07]
 [9.99999761e-01 2.39237670e-07]
 [9.99999879e-01 1.20858146e-07]
 [9.99999872e-01 1.28480155e-07]
 [9.99999423e-01 5.77428147e-07]
 [9.99999773e-01 2.26863551e-07]
 [9.99999887e-01 1.13176929e-07]
 [9.99999986e-01 1.36472599e-08]
 [9.99999988e-01 1.21352364e-08]
 [9.99999872e-01 1.28433278e-07]
 [9.99999986e-01 1.36449278e-08]
 [9.99999499e-01 5.01206556e-07]
 [9.99999745e-01 2.55457037e-07]
 [9.99999744e-01 2.56316074e-07]
 [9.99996396e-01 3.60360744e-06]
 [9.99999912e-01 8.76448509e-08]
 [9.99999871e-01 1.28559355e-07]
 [9.99999872e-01 1.28322043e-07]
 [9.99999987e-01 1.28398866e-08]
 [9.99999147e-01 8.53161400e-07]
 [9.99999390e-01 6.09744250e-07]
 [9.99999848e-01 1.52286001e-07]
 [9.99999761e-01 2.38926678e-07]
 [9.99999872e-01 1.28311623e-07]
 [9.99999761e-01 2.38914991e-07]
 [9.99999871e-01 1.28559816e-07]
 [9.99999872e-01 1.28324100e-07]
 [9.999997

In [179]:
df_test[0:1]

,選手名,チーム,防御率,試合,勝利,敗北,セーブ,ホールド,勝率,打者,...,与死球,奪三振,失点,自責点,WHIP,DIPS,完投,完封,year,沢村賞
3896,山本 由伸,オリックス,1.39,26,18,5,0,0,0.783,736,...,2,206,37,30,0.85,2.1,6,4,2021,1


ロジスティック回帰のモデルと比べると精度は明らかに良くなっている。山本由伸の沢村賞確率が約94％でしっかりと予測できている。

### LightGBMモデルの評価

訓練データとテストデータに対する正解率

In [180]:
#訓練データの正解率
print('train score : ', model_lgb.score(X_train, y_train))

train score :  1.0


In [181]:
#テストのデータの正解率
print('test score : ', model_lgb.score(X_test, y_test))

test score :  1.0


F値による評価

In [182]:
from sklearn.metrics import f1_score
f_score = f1_score(y_test,y_pred_lgb)
print(f_score)

1.0
